In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import cv2
from google.colab.patches import cv2_imshow # colab does not support cv2.imshow()

# Download & compile the YOLO Darknet network into Google Colab



In [3]:
%cd "/content/drive/MyDrive/Colab Notebooks"

/content/drive/MyDrive/Colab Notebooks


In [4]:
!git clone https://github.com/AlexeyAB/darknet

fatal: destination path 'darknet' already exists and is not an empty directory.


In [5]:
%pwd

'/content/drive/MyDrive/Colab Notebooks'

In [6]:
%cd ./darknet

/content/drive/MyDrive/Colab Notebooks/darknet


In [7]:
!chmod +rwx ./*

Open the **makefile** file and change these lines

GPU=1

CUDNN=1

OPENCV=1

*Line 20, comment*
`# ARCH= -gencode arch=compute_20,code=[sm_20,sm_21]` \ 
-gencode arch=compute_30, code=...

For K80 GPU, add in *line 61* :

`ARCH= -gencode arch=compute_37, code=sm_37`

In [8]:
!make

chmod +x *.sh


# Build the Network Config file

1.  Copy the yolov4.cfg file an create this new copy: document_yolov4.cfg
2.  Open and modify the file with these entries :

    Line 3 : subdivisions=12
    
    Line 7: width=416

    Line 8: height=416

    this should be 2000 * number of classes \
    Line 19: max_batches = 2000

    this should be 80% and 90% percent of max_batches respectively \
    Line 21: steps=1600,1800

    Line 961,1049,1137: filters=18 # (classes+5)*3
    
    Line 968,1056,1144: classes=1


# Test the YOLO network


In [9]:
# !./darknet detector test cfg/coco.data "/content/drive/MyDrive/Colab Notebooks/darknet/doc_data/yolov4.cfg"  "/content/drive/MyDrive/Colab Notebooks/darknet/doc_data/yolov4.weights"  "/content/drive/MyDrive/Colab Notebooks/darknet/doc_data/yoloimg.jpg"


# Prepare the custom model

1. Get images from any sources ...
2. Label images by using for example LabelImg (https://github.com/tzutalin/labelImg)

After labeling all your images (here i'll label only 300 images) you should have 3 kind of files:

- the images itself (jpg, png, etc.)
- a set of text file (one per image) associated with the coordinates of the boxes in the image (thes files are generated by LabelImg)
- a classes file which list the classes

Here the images are located in /content/drive/MyDrive/Colab Notebooks/darknet/doc_data/doc_images

# Configuration files
## Split the dataset (Train/Test)

1. Create one file with the list of 80% of the images (train) : 240 images in our case (document_training.txt)
2. Create one file with the list of 20% of the images (test) : 60 images in our case (document_test.txt)
3. Create a folder named doc_data into the root of darknet (/content/drive/MyDrive/Colab Notebooks/darknet/doc_data)
4. Put the two files in this directory

In [10]:
from os import listdir
from os.path import isfile, join
mydirectory = "/content/drive/MyDrive/Colab Notebooks/darknet/doc_data/doc_images"
allfiles = [f for f in listdir(mydirectory) if isfile(join(mydirectory, f))]

In [11]:
len(allfiles)

443

In [12]:
rootdir = "doc_data/doc_images/"
counter = 1
images_files = []

# put all the images files into a list
for file in allfiles:
  if (file[len(file)-3:len(file)] != "txt"):
    images_files.append(rootdir +  file)

# Set the size of the test (20%) & train data (80%)
training_limit = int(len(images_files) * 80 / 100)
test_limit = len(images_files) - training_limit

# Build the two lists
images_training_files = []
images_test_files = []
for file in images_files:
  if (counter <= training_limit):
    images_training_files.append(file)
  else:
    images_test_files.append( file)
  counter = counter + 1

# Put the two list into two files
with open('/content/drive/MyDrive/Colab Notebooks/darknet/doc_data/document_test.txt', 'w') as f:
    for item in images_test_files:
        f.write("%s\n" % item)

with open('/content/drive/MyDrive/Colab Notebooks/darknet/doc_data/document_training.txt', 'w') as f:
    for item in images_training_files:
        f.write("%s\n" % item)

# List of labels/classes
Then create a file named document.names and put the label (only one here) in there.

# Configuration bundle file
Create a file named document.data which will regroup all the configuration. Put that content inside :


```
classes = 1
valid = doc_data/document_test.txt
train = doc_data/document_training.txt
names = doc_data/document.names
backup = backup
```

# Train the model

In [13]:
%cd "/content/drive/MyDrive/Colab Notebooks/darknet"
!chmod +rwx ./*

/content/drive/MyDrive/Colab Notebooks/darknet


In [14]:
# !./darknet detector train doc_data/document.data doc_data/yolov4_custom.cfg doc_data/yolov4.conv.137 -dont_show -map

Use the command line **above** if you haven't started the training yet. It will generate the first backup file you need to train the rest of the model (ie : yolov4_custom_last.weights)

In [15]:
!./darknet detector train doc_data/document.data doc_data/yolov4_custom.cfg backup/yolov4_custom_last.weights -dont_show -map


 CUDA-version: 11010 (11020), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
 Prepare additional network for mAP calculation...
 0 : compute_capability = 750, cudnn_half = 0, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 1, batch = 12, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     64       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  64 0.354 BF
   3 route  1 		                           ->  208 x 208 x  64 
   4 conv     64       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  64 0.354 BF
   5 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   6 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   7 Shortcut Layer: 4,  wt = 0, wn = 0, outputs: 208 x 208 x  6

# Save the changes on Google Drive and unmount the directory

In [16]:
drive.flush_and_unmount()
print('All changes made in this colab session should now be visible in Drive.')

All changes made in this colab session should now be visible in Drive.
